In [1]:
from torchrl.envs import BraxWrapper
import brax.envs as envs
from Rodent_Env_Brax import Rodent

In [2]:
envs.register_environment("rodent", Rodent)

In [3]:
type(envs.get_environment("rodent"))

2024-04-04 12:49:44.170619: W external/xla/xla/service/gpu/nvptx_compiler.cc:718] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.4.131). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.


Rodent_Env_Brax.Rodent

In [4]:
import torch
device = torch.device("cuda")

In [5]:
env = BraxWrapper(envs.get_environment("rodent"), device=device)

NotImplementedError: unsupported data type <class 'tuple'>

In [36]:
env.set_seed(0)
td = env.reset()
print(td)

TensorDict(
    fields={
        done: Tensor(shape=torch.Size([1]), device=cuda:0, dtype=torch.bool, is_shared=True),
        observation: Tensor(shape=torch.Size([1260]), device=cuda:0, dtype=torch.float32, is_shared=True),
        state: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                metrics: TensorDict(
                    fields={
                        distance_from_origin: Tensor(shape=torch.Size([]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        forward_reward: Tensor(shape=torch.Size([]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        reward_alive: Tensor(shape=torch.Size([]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        reward_linvel: Tensor(shape=torch.Size([]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        reward_quadctrl: Tensor(shape=torch.Size

In [37]:
td["action"] = env.action_spec.rand()
td = env.step(td)
print(td)

TensorDict(
    fields={
        action: Tensor(shape=torch.Size([30]), device=cuda:0, dtype=torch.float32, is_shared=True),
        done: Tensor(shape=torch.Size([1]), device=cuda:0, dtype=torch.bool, is_shared=True),
        next: TensorDict(
            fields={
                done: Tensor(shape=torch.Size([1]), device=cuda:0, dtype=torch.bool, is_shared=True),
                observation: Tensor(shape=torch.Size([1260]), device=cuda:0, dtype=torch.float32, is_shared=True),
                reward: Tensor(shape=torch.Size([1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                state: TensorDict(
                    fields={
                        done: Tensor(shape=torch.Size([1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        metrics: TensorDict(
                            fields={
                                distance_from_origin: Tensor(shape=torch.Size([]), device=cuda:0, dtype=torch.float32, is_shared=True),
               

In [38]:
td = env.rand_step(td)


In [39]:
%timeit env.rand_step(td)

34.9 ms ± 876 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [40]:
b_env = Rodent()

In [30]:
import jax
import brax
key = jax.random.key(0)
state = b_env.reset(key)

In [31]:
jit_step = jax.jit(b_env.step)

In [32]:
action = jax.random.uniform(key, shape=state.pipeline_state.ctrl.shape)
state = jit_step(state, action)

In [33]:
action = jax.random.uniform(key, shape=state.pipeline_state.ctrl.shape)
%timeit jit_step(state, action)

12.4 ms ± 951 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


# testing code

In [4]:
import jax
env = Rodent()

In [5]:
key = jax.random.PRNGKey(0)
state = env.reset(key)

In [6]:
state.__dict__.keys()

dict_keys(['pipeline_state', 'obs', 'reward', 'done', 'metrics', 'info'])

In [7]:
for k in state.__dict__.keys():
    print(type(getattr(state, k)))
    

<class 'brax.mjx.base.State'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'dict'>
<class 'dict'>


In [8]:
data = state.pipeline_state
print(type(data.contact))

<class 'brax.base.Contact'>


In [9]:
contact = data.contact

In [10]:
for k in contact.__dict__.keys():
    print(type(getattr(contact, k)))

<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'jaxlib.xla_extension.ArrayImpl'>
<class 'tuple'>
<class 'jaxlib.xla_extension.ArrayImpl'>


In [11]:
contact.link_idx

(Array([-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
        -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],      dtype=int32),
 Array([10, 10, 11, 11, 11, 11, 11, 11, 14, 14, 15, 15, 15, 15, 15, 15, 24,
        24, 35, 35, 59, 59, 59, 59, 59, 59, 64, 64, 64, 64, 64, 64, 58, 63],      dtype=int32))